In [ ]:
from torchvision.datasets import ImageFolder
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = ImageFolder("data/test_loader/", transform=transform)

In [ ]:
train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.5),
     transforms.RandomRotation(24),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32
trainset = ImageFolder("data/train/", transform=train_transform)
testset = ImageFolder("data/test/", transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False)

In [ ]:
import os, shutil

In [ ]:
test_mapping = pd.DataFrame()
for img, cl in testset.imgs:
    new_img_name = str(np.random.rand())[2:]
    test_mapping=test_mapping.append({"img":new_img_name,"cl":cl},ignore_index=True)
    shutil.copy(img,f"data/test_all/{new_img_name}.JPEG")
#     print(img, cl)

In [ ]:
test_mapping["img"] = test_mapping.img + ".JPEG"

In [ ]:
test_mapping.to_csv("data/test_mapping",index=False)

# Final evaluation

In [ ]:
from torchvision.datasets import ImageFolder
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [ ]:
test_mapping = pd.read_csv("data/test_mapping")

In [ ]:
pd.DataFrame({"a":test_mapping.img, "b":np.random.randint(0,50,len(test_mapping))}).to_csv("all_pia/results/random.csv", index=False, header=None)

In [ ]:
import os
directory = "results/poniedzialek/all_pon/"
files = os.listdir(directory)

In [ ]:
results = pd.DataFrame()
for file in files:
    try:
        if file[-4:]==".csv":
            team_name = file[:-4]
        else:
            team_name = file
        predictions = pd.read_csv(directory+file, header=None)
        if len(predictions.columns)!=2:
            print(f"Warning: more columns for team {team_name}")
            pred_scores = predictions.merge(test_mapping,left_on=1,right_on="img")
            scores = pred_scores[2]
        else:
            pred_scores = predictions.merge(test_mapping,left_on=0,right_on="img")
            scores = pred_scores[1]

        accuracy = (scores == pred_scores.cl).sum()/len(test_mapping)
        print(len(scores), len(pred_scores))
        results = results.append({"team":team_name,"accuracy":accuracy},ignore_index=True)
    except:
        print(f"Error for {file}")
        continue
    print(team_name,accuracy)

In [ ]:
results.sort_values("accuracy",ascending=False)